# Flash Calculations in Pyomo

&#169; University of Notre Dame. CBE 20258. Spring 2020.

<div class="admonition seealso" name="html-admonition"> 
<p class="title"><b>Home Activity</b></p>
 You are expected to read this entire notebook before class.
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyomo.environ import *

## Learning Objectives

After studying this notebook, completing the activities, and asking questions in class, you should be able to:
* Apply the process of creating a mathematical model and implementing it in pyomo to any nonlinear optimization problem.
* Understand and interpret the solutions to a pyomo model.

## Flash Example Revisited

![flash](../../media/flash-system.png)

Let's revist the flash example. Here is the **mathematical model**:

Feed Specifications: $F = 1.0$ mol/s, $z_1$ = 0.5 mol/mol, $z_2$ = 0.5 mol/mol

Given Equilibrium Data: $K_1$ = 3 mol/mol, $K_2$ = 0.05 mol/mol

Overall Material Balance

$$F = L + V$$

Component Mass Balances

$$V y_1 + L x_1 = F z_1$$

$$V y_2 + L x_2 = F z_2$$

Thermodynamic Equilibrium

$$y_1 = K_1 x_1$$

$$y_2 = K_2 x_2$$

Summation

$$y_1 + y_2 = x_1 + x_2$$

And here is part of the **code** from The Newton's Method Notebook in Chapter 4:

In [2]:
def my_f(x):
    ''' Nonlinear system of equations in canonical form F(x) = 0
    Copied from Lecture 7.
    
    Arg:
        x: vector of variables
        
    Returns:
        r: residual, F(x)
    
    '''

    # Initialize residuals
    r = np.zeros(6)
    
    # given data
    F = 1.0
    z1 = 0.5
    z2 = 0.5
    K1 = 3
    K2 = 0.05
    
    # copy values from x to more meaningful names
    L = x[0]
    V = x[1]
    x1 = x[2]
    x2 = x[3]
    y1 = x[4]
    y2 = x[5]
    
    # equation 1: overall mass balance
    r[0] = V + L - F
    
    # equations 2 and 3: component mass balances
    r[1] = V*y1 + L*x1 - F*z1
    r[2] = V*y2 + L*x2 - F*z2
    
    # equation 4 and 5: equilibrium
    r[3] = y1 - K1*x1
    r[4] = y2 - K2*x2
    
    # equation 6: summation
    r[5] = (y1 + y2) - (x1 + x2)
    # This is known as the Rachford-Rice formulation for the summation constraint
    
    return r


<div class="admonition note" name="html-admonition"> 
<p class="title"><b>Class Activity</b></p>
 Complete the pyomo model below using the code above and solve the flash calculation.
</div>

In [3]:
## Create a concrete model
m = ConcreteModel()

## Define the objective to be a constant
m.obj = Objective(expr=1)

## Define a set for components
m.COMPS = Set(initialize=[1,2])

## given data
m.F = Param(initialize=1.0)
m.z = Param(m.COMPS, initialize={1:0.5, 2:0.5})

# Add your solution here

## Define variables
m.L = Var(initialize=0.5)
m.V = Var(initialize=0.5)
m.x = Var(m.COMPS, bounds=(0, 1.0), initialize={1:0.5, 2:0.5})
# Add your solution here

### Define equations

# equation 1: overall mass balance
m.overall_mass_balance = Constraint(expr=m.F == m.V + m.L)

# equations 2 and 3: component mass balances
def eq_comp_mass_balance(model, c):
    ''' model: Pyomo model
        c: set for components
    '''
    
    return model.V * model.y[c] + model.L * model.x[c] == model.F * model.z[c]
    
m.component_mass_balance = Constraint(m.COMPS, rule=eq_comp_mass_balance)

# equation 4 and 5: equilibrium
# Add your solution here

# equation 6: summation
# Add your solution here
# This is known as the Rachford-Rice formulation for the summation constraint

## Print model
m.pprint()

1 Set Declarations
    COMPS : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}

3 Param Declarations
    F : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :   1.0
    K : Size=2, Index=COMPS, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :     3
          2 :  0.05
    z : Size=2, Index=COMPS, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :   0.5
          2 :   0.5

4 Var Declarations
    L : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.5 :  None : False : False :  Reals
    V : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.5 :  None : False : False :  Reals
    x : Size=2, Index=COMPS
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.5 :   1.0 :

In [4]:
## Solve the model

# Specify the solver
solver = SolverFactory('ipopt')

# Solve!
results = solver.solve(m, tee = True)

    ipopt


ApplicationError: No executable found for solver 'ipopt'

In [ ]:
## Examine the solution
print("L = ",value(m.L),"mol/s")
# Add your solution here